In [ ]:
!pip install -U sagemaker

In [1]:
import sagemaker
from sagemaker.huggingface import HuggingFace

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
role = sagemaker.get_execution_role()

In [3]:
role

'arn:aws:iam::719742385994:role/SageMakerLLMRole'

In [4]:
hyperparameters = {
    "model_id": "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T",
    "epochs": 2,
    "per_device_train_batch_size": 2,
    "lr": 2e-5
}

In [10]:
estimator = HuggingFace(
    entry_point="train.py",
    source_dir="./script",
    role=role,
    transformers_version="4.36",
    pytorch_version="2.1",
    py_version="py310",
    instance_type="ml.g5.xlarge",
    instance_count=1,
    output_path="s3://llm-model-artifacts-girish/models/",
    hyperparameters=hyperparameters
)

In [11]:
estimator.fit({
    "train": "s3://llm-finetune-dataset-girish/datasets/"
})

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-pytorch-training-2025-11-19-11-16-01-374


2025-11-19 11:16:02 Starting - Starting the training job
2025-11-19 11:16:02 Pending - Training job waiting for capacity......
2025-11-19 11:16:37 Pending - Preparing the instances for training...
2025-11-19 11:17:27 Downloading - Downloading the training image........................
2025-11-19 11:21:35 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
/opt/conda/lib/python3.10/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/opt/conda/lib/python3.10/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorith

In [13]:
# estimator.latest_training_job.model_data


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ ❱ 1 estimator.latest_training_job.model_data                                                     │
│   2 # estimator.model_data                                                                       │
│   3                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: '_TrainingJob' object has no attribute 'model_data'

In [14]:
estimator.model_data

's3://llm-model-artifacts-girish/models/huggingface-pytorch-training-2025-11-19-11-16-01-374/output/model.tar.gz'

In [9]:
# predictor = huggingface_estimator.deploy(
#     initial_instance_count=1,
#     instance_type="ml.g5.xlarge"
# )

# predictor.predict({"inputs": "Explain AWS S3"})


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ ❱ 1 predictor = huggingface_estimator.deploy(                                                    │
│   2 │   initial_instance_count=1,                                                                │
│   3 │   instance_type="ml.g5.xlarge"                                                             │
│   4 )                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'huggingface_estimator' is not defined

In [ ]:
# Use this code to check all the accessible services inside your AWS Sagemaker

# from sagemaker import image_uris

# image_uris.retrieve(
#     framework="huggingface",
#     region="ap-south-1",   # change your region
#     version="4.37.0",
#     image_scope="inference"
# )

In [ ]:
# instance_type="ml.g5.xlarge"

In [ ]:
## the code for the deployment
import sagemaker
from sagemaker.huggingface import HuggingFaceModel

role = sagemaker.get_execution_role()

# model = HuggingFaceModel(
#     model_data="s3://llm-model-artifacts-girish/models/huggingface-pytorch-training-2025-11-19-11-16-01-374/output/model.tar.gz",
#     role=role,
#     transformers_version="4.37.0",
#     pytorch_version="2.1.0",
#     py_version="py310",
# )

model = HuggingFaceModel(
    model_data="s3://llm-model-artifacts-girish/models/huggingface-pytorch-training-2025-11-19-11-16-01-374/output/model.tar.gz",
    role=role,
    entry_point="inference.py",
    source_dir="inference",
    transformers_version="4.36",
    pytorch_version="2.1",
    py_version="py310"
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.xlarge",
    endpoint_name="girish-live-finetune-endpoint"
)


INFO:sagemaker:Creating model with name: huggingface-pytorch-inference-2025-11-20-11-41-09-156
INFO:sagemaker:Creating endpoint-config with name girish-live-finetune-endpoint
INFO:sagemaker:Creating endpoint with name girish-live-finetune-endpoint
ERROR:sagemaker:Please check the troubleshooting guide for common errors: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-troubleshooting.html#sagemaker-python-sdk-troubleshooting-create-endpoint


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 15>:15                                                                            │
│                                                                                                  │
│   12 │   py_version="py310",                                                                     │
│   13 )                                                                                           │
│   14                                                                                             │
│ ❱ 15 predictor = model.deploy(                                                                   │
│   16 │   initial_instance_count=1,                                                               │
│   17 │   instance_type="ml.g5.xlarge",                                                           │
│   18 │   endpoint_name="girish-live-finetune-endpoint"                                           │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/huggingface/model.p │
│ y:326 in deploy                                                                                  │
│                                                                                                  │
│   323 │   │   │   │   inference_tool=inference_tool,                                             │
│   324 │   │   │   )                                                                              │
│   325 │   │                                                                                      │
│ ❱ 326 │   │   return super(HuggingFaceModel, self).deploy(                                       │
│   327 │   │   │   initial_instance_count,                                                        │
│   328 │   │   │   instance_type,                                                                 │
│   329 │   │   │   serializer,                                                                    │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/model.py:1814 in    │
│ deploy                                                                                           │
│                                                                                                  │
│   1811 │   │   │   │   )                                                                         │
│   1812 │   │   │   │   self.sagemaker_session.update_endpoint(self.endpoint_name, endpoint_conf  │
│   1813 │   │   │   else:                                                                         │
│ ❱ 1814 │   │   │   │   self.sagemaker_session.endpoint_from_production_variants(                 │
│   1815 │   │   │   │   │   name=self.endpoint_name,                                              │
│   1816 │   │   │   │   │   production_variants=[production_variant],                             │
│   1817 │   │   │   │   │   tags=tags,                                                            │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py:6250 in  │
│ endpoint_from_production_variants                                                                │
│                                                                                                  │
│   6247 │   │   logger.info("Creating endpoint-config with name %s", name)                        │
│   6248 │   │   self.sagemaker_client.create_endpoint_config(**config_options)                    │
│   6249 │   │                                                                                     │
│ ❱ 6250 │   │   return self.create_endpoint(                

In [ ]:
## after the deployment URL will look like this
# https://runtime.sagemaker.<region>.amazonaws.com/endpoints/live-finetune-endpoint/invocations

In [ ]:
## after the deployment URL will look like this
https://runtime.sagemaker.us-east-1.amazonaws.com/endpoints/live-finetune-endpoint/invocations

In [16]:
predictor.predict({"inputs": "Explain AWS S3"})

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ ❱ 1 predictor.predict({"inputs": "Explain AWS S3"})                                              │
│   2                                                                                              │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/base_predictor.py:2 │
│ 12 in predict                                                                                    │
│                                                                                                  │
│   209 │   │   if inference_component_name:                                                       │
│   210 │   │   │   request_args["InferenceComponentName"] = inference_component_name              │
│   211 │   │                                                                                      │
│ ❱ 212 │   │   response = self.sagemaker_session.sagemaker_runtime_client.invoke_endpoint(**req   │
│   213 │   │   return self._handle_response(response)                                             │
│   214 │                                                                                          │
│   215 │   def _handle_response(self, response):                                                  │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/client.py:602 in     │
│ _api_call                                                                                        │
│                                                                                                  │
│    599 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    600 │   │   │   │   )                                                                         │
│    601 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  602 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    603 │   │                                                                                     │
│    604 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    605                                                                                           │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/context.py:123 in    │
│ wrapper                                                                                          │
│                                                                                                  │
│   120 │   │   │   with start_as_current_context():                                               │
│   121 │   │   │   │   if hook:                                                                   │
│   122 │   │   │   │   │   hook()                                                                 │
│ ❱ 123 │   │   │   │   return func(*args, **kwargs)                                               │
│   124 │   │                                                                                      │
│   125 │   │   return wrapper                                                                     │
│   126                                                                                            │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-p